<a href="https://colab.research.google.com/github/MWSO/Study_AI/blob/main/natural_language_editor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 自然言語処理実装 for Google Colab

1. 文字の扱い方について学習します。（8問）
    - 文字の加工、処理方法について学習します。
2. 正規表現の扱い方について学習します。（11問）
    - 正規表現を用いた文字の加工方法について学習します。
3. MeCabについて学習します。（6問）
    - MeCabを用いた文章の「分かち書き」、「形態素解析」について学習します。
4. Word2vecについて学習します。（12問）
    - 実際の文章データを使用して、整形からWord2vecの計算まで行います。
5. tf-idfについて学習します。（4問）
    - tf-idfの計算方法について学習します。

### 0. 事前準備

#### 0.1 必要なライブラリをインストールして下さい。
- 5〜15分程度かかることがあります。
---

In [5]:
!apt install aptitude
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3
!pip install unidic-lite
!pip install janome
!pip install gensim

Reading package lists... Done
Building dependency tree       
Reading state information... Done
aptitude is already the newest version (0.8.12-1ubuntu4).
0 upgraded, 0 newly installed, 0 to remove and 22 not upgraded.
mecab is already installed at the requested version (0.996-10build1)
libmecab-dev is already installed at the requested version (0.996-10build1)
mecab-ipadic-utf8 is already installed at the requested version (2.7.0-20070801+main-2.1)
git is already installed at the requested version (1:2.25.1-1ubuntu3.10)
make is already installed at the requested version (4.2.1-1.2)
curl is already installed at the requested version (7.68.0-1ubuntu2.16)
xz-utils is already installed at the requested version (5.2.4-1ubuntu1.1)
file is already installed at the requested version (1:5.38-4)
mecab is already installed at the requested version (0.996-10build1)
libmecab-dev is already installed at the requested version (0.996-10build1)
mecab-ipadic-utf8 is already installed at the requested ve

#### 0.2 必要なモジュールを読み込んで下さい。

In [6]:
import MeCab
import pandas as pd
import warningｓ
import os

warnings.simplefilter('ignore')

#### 0.3 必要なフォルダとファイルを準備してください。
1. 画面左側のフォルダアイコンをクリックしてください。
2. クリックしてしばらく待つと、`sample_data`というフォルダが表示されます。この状態になったらフォルダを追加できます。
3. フォルダが表示されているエリアの空白を右クリックし、「フォルダを追加」をクリックします。
4. 作成したフォルダの名前を `latest-ja-word2vec-gensim-model` に変更します。
5. ダウンロードした課題フォルダを開き `latest-ja-word2vec-gensim-model` のフォルダ内にあるファイルを、4で作成したGoogle Colab側の`latest-ja-word2vec-gensim-model` フォルダにアップロードします。(5〜15分程度かかることがあります。)
  - word2vec.gensim.model
  - word2vec.gensim.model.syn1neg.npy
  - word2vec.gensim.model.wv.syn0.npy

#### 0.4 以下のコードを実行して、学習済みモデルのファイルが存在しているかを確認して下さい。
- 存在していればTrueと出力されます
- Falseと表示された場合、0.3の手順がうまくいっていない可能性があります。
  - フォルダとファイルが正しい名前になっているか、再度確認してください。
#### ※参考サイト：「 https://aial.shiroyagi.co.jp/2017/02/japanese-word2vec-model-builder/ 」

In [7]:
trained_model_path = './latest-ja-word2vec-gensim-model/word2vec.gensim.model'
os.path.isfile(trained_model_path)

True

### 1. 文字の扱い方について

#### 1.1 「スライス」を利用して「aiueo」と言う文字を反対から出力して下さい。

In [9]:
test = "aiueo"
test[::-1]

'oeuia'

#### 1.2 「スライス」を利用して「あかさたなはまやらわ」から奇数番目の文字を出力して下さい。

In [12]:
test = "あかさたなはまやらわ"
test[::2]

'あさなまら'

#### 1.3 「スライス」を利用して「あかさたなはまやらわ」の文字を逆順にして、奇数番目の文字を出力して下さい。

In [13]:
test = "あかさたなはまやらわ"
test[::-1][::2]

'わやはたか'

#### 1.4 引数a, bを受け取り「aの時はb」という文字列を返す関数を実装し、a="ご飯", b="静かにする"として、実行結果を確認して下さい。

In [15]:
def then(a,b):
  return "{}の時は{}".format(a,b)
then("ご飯", "静かにする")


'ご飯の時は静かにする'

#### 1.5 「abcde」、「123」の文字列を別々の変数に格納し、下記の様に要素を1つずつ順番に組み合わせ、足りない部分に「zzz」という文字を入れて下さい。※itertoolsモジュールを使用して下さい。

[('a', '1'), ('b', '2'), ('c', '3'), ('d', 'zzz'), ('e', 'zzz')]

#### 1.6 「葛飾区亀有公園前派出所」の文字列から下記のタプルリストを作成して下さい。※ヒント：n-gram

[('葛', '飾'),
 ('飾', '区'),
 ('区', '亀'),
 ('亀', '有'),
 ('有', '公'),
 ('公', '園'),
 ('園', '前'),
 ('前', '派'),
 ('派', '出'),
 ('出', '所')]

#### 1.7 「葛飾区亀有公園前派出所」の文字列から下記タプルリストを作成して下さい。※ヒントn-gram

[('葛', '飾', '区'),
 ('飾', '区', '亀'),
 ('区', '亀', '有'),
 ('亀', '有', '公'),
 ('有', '公', '園'),
 ('公', '園', '前'),
 ('園', '前', '派'),
 ('前', '派', '出'),
 ('派', '出', '所')]

### 2. 正規表現について　※reモジュールを使用し、正規表現で「パターンマッチング」して課題に取り組んで下さい。スライスの使用は禁止です。

#### 2.1 reモジュールを読み込んで下さい。

In [16]:
import re

#### 2.2 電話番号「012-3456-7890」から市外局番である「012」を抜き出して下さい。

In [17]:
tel_number = "012-3456-7890"
match = re.search(r"\d+", tel_number)
print(match.group())

012


#### 2.3 文字列「axyzb」から「xyz」を抜き出して下さい。

In [18]:
string = "axyzb"
print(re.findall("a(.*)b", string))

['xyz']


####  2.4 「'Dear aaa@google.com Mr.Sato'」から「メールアドレスのみ」を出力して下さい。

In [21]:
address = 'Dear aaa@google.com Mr.Sato'
match = re.search(r"[\w.-]+@[\w.-]+", address)
print(match.group())

aaa@google.com


#### 2.5「'Dear aaa@google.com Mr.Sato'」から「メールアドレス」、「ユーザー名」、「ドメイン名」をそれぞれ出力して下さい。

In [32]:
l = 'Dear aaa@google.com Mr.Sato'
match = re.search(r'([\w.-]+)@([\w.-]+)', l)
if match:
    print(match.group())   
    print(match.group(1))  
    print(match.group(2))  

aaa@google.com
aaa
google.com


#### 2.6 「アルファベットのみ」を認識する関数を作成し、その関数に「123456」、「123a」、「abc」、「ABC」を入力し、下記結果を出力して下さい。

In [33]:
def is_alphabet(l):
  charRe = re.compile(r"[^a-zA-Z.]")
  match = charRe.search(l)
  return not bool(match)

print("123456:", is_alphabet("123456"))
print("123a:", is_alphabet("123a"))
print("abc:", is_alphabet("abc"))
print("ABC:", is_alphabet("ABC"))

123456: False
123a: False
abc: True
ABC: True


#### 2.7 アルファベットの中でも「大文字」、「小文字」を認識する関数を作成して下さい。その関数に「aaabbbc」、「aAabbcc」、「AAABBBC」を入力し、下記結果を出力して下さい。

In [34]:
def text_matching(text):
  patterns = "^[a-z]+[a-z]+$"
  if re.search(patterns, text):
    return True
  else:
    return False

print("aaabbbc:", text_matching("aaabbbc"))
print("aAabbcc:", text_matching("aAabbcc"))
print("AAABBBC:", text_matching("AAABBBC"))


aaabbbc: True
aAabbcc: False
AAABBBC: False


#### 2.8 ipアドレス「211.08.085.341」から「0」を抜いて下さい。

In [35]:
ip = "211.08.085.341"
l = re.sub("\.[0]*", ".", ip)
print(l)

211.8.85.341


#### 2.9 URL「"https://www.aaabbb/2020/01/01/%3%7%d%7/ "」から日付を下記形式で抽出して下さい。

#### 2.10 「Tから始まる文字列かどうか」を判定する関数を作成し、「"Takashi"」,「"Noriko"」の文字列を入力した結果を出力して下さい。

#### 2.11 空白を削除する関数を作成し、「"aaa  bbb       ccc"」の結果を出力して下さい。

### 3. Mecabの使い方について

#### 3.1 「すももももももももうのうち」を分かち書きして下さい。

In [37]:
mecab = MeCab.Tagger("-Owakati")
text = mecab.parse("すもももももももものうち")
print(text)

すもも も もも も もも の うち 



#### 3.2「あの花はとても美しい」を形態素解析して下さい。

In [41]:
mecab = MeCab.Tagger("-chasen")
text = "あの花はとても美しい"
print(mecab.parse(text))

あの	アノ	アノ	あの	感動詞-フィラー			0
花	ハナ	ハナ	花	名詞-普通名詞-一般			2
は	ワ	ハ	は	助詞-係助詞			
とても	トテモ	トテモ	迚も	副詞			0
美しい	ウツクシー	ウツクシイ	美しい	形容詞-一般	形容詞	終止形-一般	4
EOS



#### 3.3 「すもももももももものうち」という文章を形態素解析して、結果を確認して下さい。

In [42]:
text = "すもももももももものうち"
mecab = MeCab.Tagger("-chasen")
print(mecab.parse(text))

すもも	スモモ	スモモ	李	名詞-普通名詞-一般			0
も	モ	モ	も	助詞-係助詞			
もも	モモ	モモ	桃	名詞-普通名詞-一般			0
も	モ	モ	も	助詞-係助詞			
もも	モモ	モモ	桃	名詞-普通名詞-一般			0
の	ノ	ノ	の	助詞-格助詞			
うち	ウチ	ウチ	内	名詞-普通名詞-副詞可能			0
EOS



#### 3.4 「すもももももももものうち」という文章が取りうるすべての形態素を表示して下さい。

In [43]:
mecab = MeCab.Tagger("-a --all-morphs")
text = mecab.parse("すもももももももものうち")
print(text)

すもも	スモモ	スモモ	李	名詞-普通名詞-一般			0
すも	スモ	スム	澄む	動詞-一般	五段-マ行	意志推量形	1
すも	スモ	スム	済む	動詞-一般	五段-マ行	意志推量形	1
すも	スモ	スム	住む	動詞-一般	五段-マ行	意志推量形	1
す	ス	デス	です	助動詞	助動詞-デス	終止形-一般	
す	ス	セル	せる	助動詞	文語下二段-サ行	終止形-一般	
す	ス	スル	為る	動詞-非自立可能	サ行変格	連体形-省略	0
す	ス	スル	為る	動詞-非自立可能	文語サ行変格	終止形-一般	1
す	ス	ス	酢	名詞-普通名詞-一般			1
す	ス	ス	素	接頭辞			
す	ス	ス	巣	名詞-普通名詞-一般			0,1
もも	モモ	モモ	腿	名詞-普通名詞-一般			1
もも	モモ	モモ	桃	名詞-普通名詞-一般			0
もも	モモ	モム	揉む	動詞-一般	五段-マ行	意志推量形	0
も	モ	モウ	もう	副詞			1
も	モ	モ	モ	記号-一般			1
も	モ	モ	も	助詞-係助詞			
もも	モモ	モモ	腿	名詞-普通名詞-一般			1
もも	モモ	モモ	桃	名詞-普通名詞-一般			0
もも	モモ	モム	揉む	動詞-一般	五段-マ行	意志推量形	0
も	モ	モウ	もう	副詞			1
も	モ	モ	モ	記号-一般			1
も	モ	モ	も	助詞-係助詞			
もも	モモ	モモ	腿	名詞-普通名詞-一般			1
もも	モモ	モモ	桃	名詞-普通名詞-一般			0
もも	モモ	モム	揉む	動詞-一般	五段-マ行	意志推量形	0
も	モ	モウ	もう	副詞			1
も	モ	モ	モ	記号-一般			1
も	モ	モ	も	助詞-係助詞			
もも	モモ	モモ	腿	名詞-普通名詞-一般			1
もも	モモ	モモ	桃	名詞-普通名詞-一般			0
もも	モモ	モム	揉む	動詞-一般	五段-マ行	意志推量形	0
も	モ	モウ	もう	副詞			1
も	モ	モ	モ	記号-一般			1
も	モ	モ	も	助詞-係助詞			
もも	モモ	モモ	腿	名詞-普通名詞-一般			1
もも	モモ	モモ	桃	名詞-普通名詞-一般			0
もも	モモ	モム	揉む	動詞-一般	五段-マ行	意志推量形	0
も	モ	モウ	もう	副詞			1
も	モ	モ	モ	記号-一般			1
も	モ	モ	も	助詞-係助詞			

#### 3.5下記文章を形態素解析して、「名詞」をリスト化して下さい。「ウィキペディアは、信頼されるフリーなオンライン百科事典、それも質・量ともに史上最大の百科事典を、共同作業で創り上げることを目的とするプロジェクト、およびその成果である百科事典本体です。」

In [49]:
sentence = """ウィキペディアは、信頼されるフリーなオンライン百科事典、それも質・量ともに史上最大の百科事典を、
共同作業で創り上げることを目的とするプロジェクト、およびその成果である百科事典本体です。"""

t = MeCab.Tagger("-chasen")
t.parse('')
m = t.parseToNode(sentence)
keywords = []
while m:
  if m.feature.split(",")[0] == '名詞':
    keywords.append(m.surface)
    m = m.next
print(keywords)

KeyboardInterrupt: ignored

#### 3.6「3.5」の文章の「名詞」、「動詞」、「助動詞」の数をカウントして下さい。

In [48]:
sentence = """ウィキペディアは、信頼されるフリーなオンライン百科事典、それも質・量ともに史上最大の百科事典を、
共同作業で創り上げることを目的とするプロジェクト、およびその成果である百科事典本体です。"""

t = MeCab.Tagger("-chasen")
t.parse('')
m = t.parseToNode(sentence)
keywords_info = []
while m:
  hinsi = m.feature.split(',')[0]
  if hinsi in ["名詞", "動詞", "助動詞"]:
    keywords_info.append(hinsi)
  m = m.next
pd.Series(keywords_info).value_counts()

名詞     23
動詞      4
助動詞     4
dtype: int64

### 4. Word to vecについて

#### 4.0 事前準備：下記コードを読み込んで下さい。

In [64]:
import urllib.request
import zipfile

URL = 'https://www.aozora.gr.jp/cards/000148/files/789_ruby_5639.zip'
zip = '789_ruby_5639.zip'

urllib.request.urlretrieve(URL, zip)

with zipfile.ZipFile(zip, 'r') as myzip:
    myzip.extractall()
    for myfile in myzip.infolist():
        filename = myfile.filename
        with open(filename, encoding='sjis') as file:
            text = file.read()

from janome.tokenizer import Tokenizer
# 1万文字まで「text」変数に格納する
text = text[:10000]
text

'吾輩は猫である\n夏目漱石\n\n-------------------------------------------------------\n【テキスト中に現れる記号について】\n\n《》：ルビ\n（例）吾輩《わがはい》\n\n｜：ルビの付く文字列の始まりを特定する記号\n（例）一番｜獰悪《どうあく》\n\n［＃］：入力者注\u3000主に外字の説明や、傍点の位置の指定\n\u3000\u3000\u3000（数字は、JIS X 0213の面区点番号またはUnicode、底本のページと行数）\n（例）※［＃「言＋墟のつくり」、第4水準2-88-74］\n\n〔〕：アクセント分解された欧文をかこむ\n（例）〔Quid aliud est mulier nisi amicitiae& inimica〕\nアクセント分解についての詳細は下記URLを参照してください\nhttp://www.aozora.gr.jp/accent_separation.html\n-------------------------------------------------------\n\n［＃８字下げ］一［＃「一」は中見出し］\n\n\u3000吾輩《わがはい》は猫である。名前はまだ無い。\n\u3000どこで生れたかとんと見当《けんとう》がつかぬ。何でも薄暗いじめじめした所でニャーニャー泣いていた事だけは記憶している。吾輩はここで始めて人間というものを見た。しかもあとで聞くとそれは書生という人間中で一番｜獰悪《どうあく》な種族であったそうだ。この書生というのは時々我々を捕《つかま》えて煮《に》て食うという話である。しかしその当時は何という考もなかったから別段恐しいとも思わなかった。ただ彼の掌《てのひら》に載せられてスーと持ち上げられた時何だかフワフワした感じがあったばかりである。掌の上で少し落ちついて書生の顔を見たのがいわゆる人間というものの見始《みはじめ》であろう。この時妙なものだと思った感じが今でも残っている。第一毛をもって装飾されべきはずの顔がつるつるしてまるで薬缶《やかん》だ。その後《ご》猫にもだいぶ逢《あ》ったがこんな片輪《かたわ》には一度も出会《でく》わした事がない。のみならず顔の真中があまりに突起している。そうしてその穴の中から時々ぷうぷうと煙《けむり》を吹く

#### 4.1 textからヘッダー（冒頭の------で囲まれている部分）を削除して下さい。

In [65]:
text = re.split("\-{5,}", text)[2]
text

'\n\n［＃８字下げ］一［＃「一」は中見出し］\n\n\u3000吾輩《わがはい》は猫である。名前はまだ無い。\n\u3000どこで生れたかとんと見当《けんとう》がつかぬ。何でも薄暗いじめじめした所でニャーニャー泣いていた事だけは記憶している。吾輩はここで始めて人間というものを見た。しかもあとで聞くとそれは書生という人間中で一番｜獰悪《どうあく》な種族であったそうだ。この書生というのは時々我々を捕《つかま》えて煮《に》て食うという話である。しかしその当時は何という考もなかったから別段恐しいとも思わなかった。ただ彼の掌《てのひら》に載せられてスーと持ち上げられた時何だかフワフワした感じがあったばかりである。掌の上で少し落ちついて書生の顔を見たのがいわゆる人間というものの見始《みはじめ》であろう。この時妙なものだと思った感じが今でも残っている。第一毛をもって装飾されべきはずの顔がつるつるしてまるで薬缶《やかん》だ。その後《ご》猫にもだいぶ逢《あ》ったがこんな片輪《かたわ》には一度も出会《でく》わした事がない。のみならず顔の真中があまりに突起している。そうしてその穴の中から時々ぷうぷうと煙《けむり》を吹く。どうも咽《む》せぽくて実に弱った。これが人間の飲む煙草《たばこ》というものである事はようやくこの頃知った。\n\u3000この書生の掌の裏《うち》でしばらくはよい心持に坐っておったが、しばらくすると非常な速力で運転し始めた。書生が動くのか自分だけが動くのか分らないが無暗《むやみ》に眼が廻る。胸が悪くなる。到底《とうてい》助からないと思っていると、どさりと音がして眼から火が出た。それまでは記憶しているがあとは何の事やらいくら考え出そうとしても分らない。\n\u3000ふと気が付いて見ると書生はいない。たくさんおった兄弟が一｜疋《ぴき》も見えぬ。肝心《かんじん》の母親さえ姿を隠してしまった。その上｜今《いま》までの所とは違って無暗《むやみ》に明るい。眼を明いていられぬくらいだ。はてな何でも容子《ようす》がおかしいと、のそのそ這《は》い出して見ると非常に痛い。吾輩は藁《わら》の上から急に笹原の中へ棄てられたのである。\n\u3000ようやくの思いで笹原を這い出すと向うに大きな池がある。吾輩は池の前に坐ってどうしたらよかろうと考えて見た。別にこれという分別《ふんべつ》も出ない

#### 4.2 textから「《》に入る文字（ルビ）」を《》ごと削除して下さい。

In [66]:
text = re.sub("《.+?》", "", text)
text

'\n\n［＃８字下げ］一［＃「一」は中見出し］\n\n\u3000吾輩は猫である。名前はまだ無い。\n\u3000どこで生れたかとんと見当がつかぬ。何でも薄暗いじめじめした所でニャーニャー泣いていた事だけは記憶している。吾輩はここで始めて人間というものを見た。しかもあとで聞くとそれは書生という人間中で一番｜獰悪な種族であったそうだ。この書生というのは時々我々を捕えて煮て食うという話である。しかしその当時は何という考もなかったから別段恐しいとも思わなかった。ただ彼の掌に載せられてスーと持ち上げられた時何だかフワフワした感じがあったばかりである。掌の上で少し落ちついて書生の顔を見たのがいわゆる人間というものの見始であろう。この時妙なものだと思った感じが今でも残っている。第一毛をもって装飾されべきはずの顔がつるつるしてまるで薬缶だ。その後猫にもだいぶ逢ったがこんな片輪には一度も出会わした事がない。のみならず顔の真中があまりに突起している。そうしてその穴の中から時々ぷうぷうと煙を吹く。どうも咽せぽくて実に弱った。これが人間の飲む煙草というものである事はようやくこの頃知った。\n\u3000この書生の掌の裏でしばらくはよい心持に坐っておったが、しばらくすると非常な速力で運転し始めた。書生が動くのか自分だけが動くのか分らないが無暗に眼が廻る。胸が悪くなる。到底助からないと思っていると、どさりと音がして眼から火が出た。それまでは記憶しているがあとは何の事やらいくら考え出そうとしても分らない。\n\u3000ふと気が付いて見ると書生はいない。たくさんおった兄弟が一｜疋も見えぬ。肝心の母親さえ姿を隠してしまった。その上｜今までの所とは違って無暗に明るい。眼を明いていられぬくらいだ。はてな何でも容子がおかしいと、のそのそ這い出して見ると非常に痛い。吾輩は藁の上から急に笹原の中へ棄てられたのである。\n\u3000ようやくの思いで笹原を這い出すと向うに大きな池がある。吾輩は池の前に坐ってどうしたらよかろうと考えて見た。別にこれという分別も出ない。しばらくして泣いたら書生がまた迎に来てくれるかと考え付いた。ニャー、ニャーと試みにやって見たが誰も来ない。そのうち池の上をさらさらと風が渡って日が暮れかかる。腹が非常に減って来た。泣きたくても声が出ない。仕方がない、何でもよいから食物のある所ま

#### 4.3 textから「[]に入る文字」と[]を削除して下さい。

In [83]:
text = re.sub('［＃.+?］', "", text)
text

'［８字下げ］一［「一」は中見出し］吾輩は猫である。名前はまだ無い。どこで生れたかとんと見当がつかぬ。何でも薄暗いじめじめした所でニャーニャー泣いていた事だけは記憶している。吾輩はここで始めて人間というものを見た。しかもあとで聞くとそれは書生という人間中で一番｜獰悪な種族であったそうだ。この書生というのは時々我々を捕えて煮て食うという話である。しかしその当時は何という考もなかったから別段恐しいとも思わなかった。ただ彼の掌に載せられてスーと持ち上げられた時何だかフワフワした感じがあったばかりである。掌の上で少し落ちついて書生の顔を見たのがいわゆる人間というものの見始であろう。この時妙なものだと思った感じが今でも残っている。第一毛をもって装飾されべきはずの顔がつるつるしてまるで薬缶だ。その後猫にもだいぶ逢ったがこんな片輪には一度も出会わした事がない。のみならず顔の真中があまりに突起している。そうしてその穴の中から時々ぷうぷうと煙を吹く。どうも咽せぽくて実に弱った。これが人間の飲む煙草というものである事はようやくこの頃知った。この書生の掌の裏でしばらくはよい心持に坐っておったが、しばらくすると非常な速力で運転し始めた。書生が動くのか自分だけが動くのか分らないが無暗に眼が廻る。胸が悪くなる。到底助からないと思っていると、どさりと音がして眼から火が出た。それまでは記憶しているがあとは何の事やらいくら考え出そうとしても分らない。ふと気が付いて見ると書生はいない。たくさんおった兄弟が一｜疋も見えぬ。肝心の母親さえ姿を隠してしまった。その上｜今までの所とは違って無暗に明るい。眼を明いていられぬくらいだ。はてな何でも容子がおかしいと、のそのそ這い出して見ると非常に痛い。吾輩は藁の上から急に笹原の中へ棄てられたのである。ようやくの思いで笹原を這い出すと向うに大きな池がある。吾輩は池の前に坐ってどうしたらよかろうと考えて見た。別にこれという分別も出ない。しばらくして泣いたら書生がまた迎に来てくれるかと考え付いた。ニャー、ニャーと試みにやって見たが誰も来ない。そのうち池の上をさらさらと風が渡って日が暮れかかる。腹が非常に減って来た。泣きたくても声が出ない。仕方がない、何でもよいから食物のある所まであるこうと決心をしてそろりそろりと池を左りに廻り始めた。どうも非常に苦しい。そこを我慢して無理

#### 4.4 textから「空行」を削除して下さい。

In [68]:
text = re.sub("\n\n", "\n", text)
text = re.sub("\n\u3000", "", text)
text = re.sub("\n", "", text)
text

'［８字下げ］一［「一」は中見出し］吾輩は猫である。名前はまだ無い。どこで生れたかとんと見当がつかぬ。何でも薄暗いじめじめした所でニャーニャー泣いていた事だけは記憶している。吾輩はここで始めて人間というものを見た。しかもあとで聞くとそれは書生という人間中で一番｜獰悪な種族であったそうだ。この書生というのは時々我々を捕えて煮て食うという話である。しかしその当時は何という考もなかったから別段恐しいとも思わなかった。ただ彼の掌に載せられてスーと持ち上げられた時何だかフワフワした感じがあったばかりである。掌の上で少し落ちついて書生の顔を見たのがいわゆる人間というものの見始であろう。この時妙なものだと思った感じが今でも残っている。第一毛をもって装飾されべきはずの顔がつるつるしてまるで薬缶だ。その後猫にもだいぶ逢ったがこんな片輪には一度も出会わした事がない。のみならず顔の真中があまりに突起している。そうしてその穴の中から時々ぷうぷうと煙を吹く。どうも咽せぽくて実に弱った。これが人間の飲む煙草というものである事はようやくこの頃知った。この書生の掌の裏でしばらくはよい心持に坐っておったが、しばらくすると非常な速力で運転し始めた。書生が動くのか自分だけが動くのか分らないが無暗に眼が廻る。胸が悪くなる。到底助からないと思っていると、どさりと音がして眼から火が出た。それまでは記憶しているがあとは何の事やらいくら考え出そうとしても分らない。ふと気が付いて見ると書生はいない。たくさんおった兄弟が一｜疋も見えぬ。肝心の母親さえ姿を隠してしまった。その上｜今までの所とは違って無暗に明るい。眼を明いていられぬくらいだ。はてな何でも容子がおかしいと、のそのそ這い出して見ると非常に痛い。吾輩は藁の上から急に笹原の中へ棄てられたのである。ようやくの思いで笹原を這い出すと向うに大きな池がある。吾輩は池の前に坐ってどうしたらよかろうと考えて見た。別にこれという分別も出ない。しばらくして泣いたら書生がまた迎に来てくれるかと考え付いた。ニャー、ニャーと試みにやって見たが誰も来ない。そのうち池の上をさらさらと風が渡って日が暮れかかる。腹が非常に減って来た。泣きたくても声が出ない。仕方がない、何でもよいから食物のある所まであるこうと決心をしてそろりそろりと池を左りに廻り始めた。どうも非常に苦しい。そこを我慢して無理

#### 4.5 Tokenizerインスタンスを作成して下さい。

In [69]:
t = Tokenizer()
t

#### 4.6 形態素解析をして、名詞、動詞、形容詞のみを配列で抽出する関数を作成し、「明日は良い天気になる可能性が非常に高い」と言う文字列を関数に入力した結果を出力して下さい。

In [71]:
def words(text):
  tokens = t.tokenize(text)
  return [token.base_form for token in tokens
          if token.part_of_speech.split(",")[0] in ["名詞", "動詞", "形容詞"]]

ret = words("明日は良い天気になる可能性が非常に高い。")
for word in ret:
  print(word)

明日
良い
天気
なる
可能
性
非常
高い


#### 4.7 全体のテキストを句点('。')で区切った配列にし、上から10要素を出力して下さい。

In [72]:
sentences = text.split("。")
sentences[:10]

['［８字下げ］一［「一」は中見出し］吾輩は猫である',
 '名前はまだ無い',
 'どこで生れたかとんと見当がつかぬ',
 '何でも薄暗いじめじめした所でニャーニャー泣いていた事だけは記憶している',
 '吾輩はここで始めて人間というものを見た',
 'しかもあとで聞くとそれは書生という人間中で一番｜獰悪な種族であったそうだ',
 'この書生というのは時々我々を捕えて煮て食うという話である',
 'しかしその当時は何という考もなかったから別段恐しいとも思わなかった',
 'ただ彼の掌に載せられてスーと持ち上げられた時何だかフワフワした感じがあったばかりである',
 '掌の上で少し落ちついて書生の顔を見たのがいわゆる人間というものの見始であろう']

#### 4.8 それぞれの文章を単語リストに変換する処理を作成し、1文の要素を出力して下さい。

In [79]:
word_list = [words(sentence) for sentence in sentences]
for word in word_list[0]:
    print(word)

８
字
下げ
一
一
見出し
吾輩
猫


#### 4.9 word2vecをインストールして、word2vecのオプションを「size=100,min_count=5,window=5,iter=100」で設定し、学習を行って下さい。
 - size: 圧縮次元数
 - min_count: 出現頻度の低いものをカットする
 - window: 前後の単語を拾う際の窓の広さを決める
 - iter: 機械学習の繰り返し回数(デフォルト:5)十分学習できていないときにこの値を調整する

In [84]:
from gensim.models import word2vec
model = word2vec.Word2Vec(
    word_list,
    size=100,
    min_count=5,
    window=5,
    iter=100
  )

#### 4.10 「猫」という単語のベクトルを出力して下さい。

In [88]:
print(model.__dict__['wv']['猫'])

[-0.05019522 -0.11788126  0.11341928 -0.02521871  0.10513197  0.16733667
 -0.08366071 -0.07848777 -0.32286796  0.00164594  0.16380459 -0.32790825
  0.25755835 -0.02599763 -0.37022865 -0.22791801  0.08074469  0.02641271
 -0.04862097 -0.01445318 -0.20348355 -0.22092909  0.18613824  0.0201055
  0.2149427  -0.14443293  0.10658211  0.02015954  0.03569651  0.12405839
  0.1363064  -0.17204905 -0.25229526 -0.00674781 -0.03240409 -0.16794029
 -0.04240664 -0.02067936  0.25649095  0.02262579 -0.0203278  -0.04913462
  0.21264715 -0.11639852 -0.04152756  0.00563067  0.04270809  0.04727034
 -0.03996442  0.20006457  0.00124388 -0.05632925 -0.12795286  0.10979982
  0.048687    0.07415409 -0.15888982  0.00329693  0.02291731  0.13039367
 -0.13694276  0.22068606 -0.2023187   0.232711    0.05333151  0.08618946
  0.09587457  0.07937435 -0.10134506 -0.24961351  0.00296368 -0.03892026
 -0.10089226  0.03101641  0.02530717 -0.03989419  0.06050354  0.41326627
 -0.03747088  0.0559581  -0.15244704  0.17081204 -0.

#### 4.11 「猫」と類似度の高い語句を算出して下さい。※結果が一致する必要は有りません。今回は訓練データを制限しているので、精度は高く有りません。

In [89]:
ret = model.wv.most_similar(positive=["猫"])
for item in ret:
  print(item[0], item[1])

云う 0.999435544013977
見える 0.9994165897369385
方 0.9994142651557922
所 0.999405026435852
する 0.9994033575057983
なる 0.9993908405303955
彼 0.9993870258331299
人 0.9993773698806763
出す 0.9993462562561035
来る 0.999344527721405


#### 4.12 今回訓練データを制限したため、精度はよく有りませんでしたが、他の学習済みモデルを用いてみましょう。

#### 4.12.0 事前準備

In [90]:
from gensim.models.word2vec import Word2Vec

trained_model_path = './latest-ja-word2vec-gensim-model/word2vec.gensim.model'
trained_model = Word2Vec.load(trained_model_path)

#### 4.12.1 trained_modelを使用して、「猫」と類似度の高い語句を算出して下さい。

In [91]:
trained_model.most_similar("猫")

[('ネコ', 0.8973475694656372),
 ('ウサギ', 0.8950017094612122),
 ('犬', 0.8777391314506531),
 ('タヌキ', 0.8691234588623047),
 ('小鳥', 0.8644418120384216),
 ('クマ', 0.8631638288497925),
 ('赤ちゃん', 0.8595046401023865),
 ('ネズミ', 0.8567873239517212),
 ('飼い', 0.8533831238746643),
 ('黒猫', 0.853003978729248)]

### 5. tf-idf計算方法

#### 5.0 事前準備

In [92]:
corpus = [
    'This is the first document.',
    'This document is the second document.',
    'And this is the third one.',
    'Is this the first document?'
]

#### 5.1 TfidfVectorizerを「sklearn」から探して読み込んで下さい。

In [93]:
from sklearn.feature_extraction.text import TfidfVectorizer
TfidfVectorizer

sklearn.feature_extraction.text.TfidfVectorizer

#### 5.2 tfidfの転置行列を作成し、中身を表示して下さい。

In [99]:
tfidf = TfidfVectorizer()
x = tfidf.fit_transform(corpus)
print(x)

  (0, 1)	0.46979138557992045
  (0, 2)	0.5802858236844359
  (0, 6)	0.38408524091481483
  (0, 3)	0.38408524091481483
  (0, 8)	0.38408524091481483
  (1, 5)	0.5386476208856763
  (1, 1)	0.6876235979836938
  (1, 6)	0.281088674033753
  (1, 3)	0.281088674033753
  (1, 8)	0.281088674033753
  (2, 4)	0.511848512707169
  (2, 7)	0.511848512707169
  (2, 0)	0.511848512707169
  (2, 6)	0.267103787642168
  (2, 3)	0.267103787642168
  (2, 8)	0.267103787642168
  (3, 1)	0.46979138557992045
  (3, 2)	0.5802858236844359
  (3, 6)	0.38408524091481483
  (3, 3)	0.38408524091481483
  (3, 8)	0.38408524091481483


#### 5.3 tfidfの値を文章毎に算出して下さい。

In [101]:
tfidf_value = pd.DataFrame(x.toarray(), columns=tfidf.get_feature_names_out())
print(tfidf_value)

        and  document     first        is       one    second       the  \
0  0.000000  0.469791  0.580286  0.384085  0.000000  0.000000  0.384085   
1  0.000000  0.687624  0.000000  0.281089  0.000000  0.538648  0.281089   
2  0.511849  0.000000  0.000000  0.267104  0.511849  0.000000  0.267104   
3  0.000000  0.469791  0.580286  0.384085  0.000000  0.000000  0.384085   

      third      this  
0  0.000000  0.384085  
1  0.000000  0.281089  
2  0.511849  0.267104  
3  0.000000  0.384085  


#### 5.4 各文章の「tf-idf」を計算し、行列形式で文章類似度を算出して下さい。※cosine_similarityを使用して下さい。

In [100]:
from sklearn.metrics.pairwise import cosine_similarity
print(cosine_similarity(x))

[[1.         0.64692568 0.30777187 1.        ]
 [0.64692568 1.         0.22523955 0.64692568]
 [0.30777187 0.22523955 1.         0.30777187]
 [1.         0.64692568 0.30777187 1.        ]]
